In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
#Params
kld_reg= 1
adl_reg=1

fdim=16
zdim=16
sigma=1.3
past_length=8
future_length=12
data_scale=170
enc_past_size=(past_length*2,512,256,fdim)
enc_dest_size=(2,8,16,fdim)
enc_latent_size=(2*fdim,8,50,2*zdim)
dec_size=(fdim + zdim,1024,512,1024,2)
predictor_size=(2*fdim,1024,512,256,2*(future_length-1))
learning_rate=0.0003

In [3]:
def loadData(file_path: str):
  npz = np.load(file_path, allow_pickle=True)
  return npz['observations'], npz['obs_speed'], npz['targets'], npz[
      'target_speed'], npz['mean'], npz['std']

In [4]:
class Dense(tf.Module):
  def __init__(self, input_dim, output_size, name=None):
    super(Dense, self).__init__(name=name)
    self.w = tf.Variable(tf.random.uniform([input_dim, output_size],-(1.0/input_dim)**0.5,(1.0/input_dim)**0.5 ),name='w',dtype=tf.float32)
    self.b = tf.Variable(tf.random.uniform([output_size],-(1.0/input_dim)**0.5,(1.0/input_dim)**0.5 ), name='b',dtype=tf.float32)
  def __call__(self, x):
    x = tf.constant(x,dtype=tf.float32)
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

class FullyConnectedNeuralNet(tf.Module):
  def __init__(self,sizes, name=None):
    super(FullyConnectedNeuralNet, self).__init__(name=name)
    self.layers = []
    with self.name_scope:
      for i in range(len(sizes)-1):
        self.layers.append(Dense(input_dim=sizes[i], output_size=sizes[i+1]))
  @tf.Module.with_name_scope
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

In [5]:
class MainModel(tf.Module):
  def __init__(self,name=None):
    super(MainModel, self).__init__(name=name)

    self.zdim = zdim
    self.sigma = sigma

    self.pastEncoder = FullyConnectedNeuralNet(enc_past_size)

    self.destEncoder = FullyConnectedNeuralNet(enc_dest_size)

    self.latentDistributionEncoder = FullyConnectedNeuralNet(enc_latent_size)

    self.latentDistributionDecoder = FullyConnectedNeuralNet(dec_size)

    self.predictorNetwork = FullyConnectedNeuralNet(predictor_size)

  def forward(self, x, dest = []):

    if len(dest):
        self.training=True
    else:
        self.training=False
        
    # encode
    traj_past_ftr = self.pastEncoder(x)
    #print(f"ftraj max {ftraj.numpy().max()}")
    if not self.training:
        z = tf.random.normal((x.shape[0], self.zdim),0,self.sigma)

    else:
        dest_ftr = self.destEncoder(dest)
        #print(f"dest_features Max {dest_features.numpy().max()}")

        concat_ftr = tf.concat((traj_past_ftr, dest_ftr), axis = 1)
        latent =  self.latentDistributionEncoder(concat_ftr)
        mu = latent[:, 0:self.zdim] # 2-d array
        logvar = latent[:, self.zdim:] # 2-d array

        var = tf.math.exp(logvar*0.5)
        #print(f"var {var}")
        eps = tf.random.normal(var.shape)
        z = eps*var + mu
        #print(f"z -> {z}")


    latentDistributionDecoder_input = tf.concat((traj_past_ftr, z), axis = 1)
    generated_dest = self.latentDistributionDecoder(latentDistributionDecoder_input)
    
    if self.training:
        generated_dest_ftr = self.destEncoder(generated_dest)
        prediction_ftr = tf.concat((traj_past_ftr, generated_dest_ftr), axis = 1)
        pred_future = self.predictorNetwork(prediction_ftr)
        
        return (generated_dest, mu, logvar, pred_future)
    else:
        return generated_dest

  def predict(self, past, generated_dest):
        
    traj_past_ftr = self.pastEncoder(past)
    generated_dest_ftr = self.destEncoder(generated_dest)
    prediction_ftr = tf.concat((traj_past_ftr, generated_dest_ftr), axis = 1)
    future_traj = self.predictorNetwork(prediction_ftr)
    return future_traj

In [6]:
def calculate_loss(dest, dest_rec, mean, log_var, future, future_rec):
    
    rcl = tf.math.reduce_mean(tf.keras.metrics.mean_squared_error(dest, dest_rec))
    adl = tf.math.reduce_mean(tf.keras.metrics.mean_squared_error(future, future_rec))

    kld = -0.5 * tf.math.reduce_sum(1 + log_var - mean**2 - tf.math.exp(log_var))

    return rcl, kld, adl

In [7]:
def next_batch(X,batchSize):
    start = random.randint(0, len(X)-batchSize)
    return X[start:start+batchSize]

In [8]:
def train(trajx,model,optimizer):
    train_loss = 0
    total_rcl, total_kld, total_adl = 0, 0, 0
    
    traj = trajx - trajx[:, :1, :]
    traj *= data_scale

    x = traj[:, :past_length, :]
    y = traj[:, past_length:, :]

    x = x.reshape(-1, x.shape[1]*x.shape[2]) # (x,y,x,y ... )
    dest = y[:, -1, :]
    future = y[:, :-1, :].reshape(y.shape[0],-1)
                
    #x.astype(np.float64)
    #print(f"x max -> {x.max()}")

    with tf.GradientTape() as tape:
        x=tf.constant(x,dtype=tf.float32)
        tape.watch(x)
        dest_rec, mu, var, future_rec = model.forward(x, dest=dest)
        #print(f"dest_recon {dest_recon}")
        #print(f"mu {mu}")
        #print(f"var {var}")
        #print(f"interpolated_future {interpolated_future}")
                    
        rcl, kld, adl = calculate_loss(dest, dest_rec, mu, var, future, future_rec)
                    
        loss = rcl + kld * kld_reg + adl * adl_reg
        grad_sub = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grad_sub, model.trainable_variables))
                    
        #print(f"total Loss {loss}")
        #print(f"rcl Loss {rcl}")
        #print(f"kld Loss {kld}")
        #print(f"adl Loss {adl}")
    train_loss+=loss
    total_rcl+=rcl
    total_kld+=kld
    total_adl+=adl
    return train_loss, total_rcl, total_kld, total_adl
                

In [9]:
def test(trajx, model, best_of_n = 1):
    
    traj = trajx - trajx[:, :1, :]
    traj *= data_scale

    x = traj[:, :past_length, :]
    y = traj[:, past_length:, :]

    x = x.reshape(-1, x.shape[1]*x.shape[2])

    dest = y[:, -1, :]
    
    destination_errors = []
    dectination_recs = []
    
    for _ in range(best_of_n):
        x=tf.constant(x,dtype=tf.float32)
        dest_rec = model.forward(x)
        dectination_recs.append(np.array(dest_rec))

        error = np.linalg.norm(dest_rec - dest, axis = 1)
        destination_errors.append(error)

    destination_errors = np.array(destination_errors)
    dectination_recs = np.array(dectination_recs)
    # average error
    avg_dest_error = np.mean(destination_errors)

    indices = np.argmin(destination_errors, axis = 0)

    best_dest = dectination_recs[indices,np.arange(x.shape[0]),  :]

    # taking the minimum error out of all guess
    dest_error = np.mean(np.min(destination_errors, axis = 0))

    future_dest = model.predict(x, best_dest)
    # final overall prediction
    predicted_future = np.concatenate((future_dest, best_dest), axis = 1)
    predicted_future = np.reshape(predicted_future, (-1, future_length, 2))
    # ADE error
    overall_error = np.mean(np.linalg.norm(y - predicted_future, axis = 2))

    overall_error /= data_scale
    dest_error /= data_scale
    avg_dest_error /= data_scale
    #print('Test time error in destination best: {:0.3f} and mean: {:0.3f}'.format(dest_error, avg_dest_error))
    #print('Test time error overall (ADE) best: {:0.3f}'.format(overall_error))

    return overall_error, dest_error, avg_dest_error

            

In [10]:
def run_train():
    observations, _, targets, _, _, _ = loadData('./data/eth/eth_train.npz')
    train_dataset = np.concatenate([observations, targets], axis=1)
    observations, _, targets, _, _, _ = loadData('./data/eth/eth_test.npz')
    test_dataset = np.concatenate([observations, targets], axis=1)
    
    epochs = 8
    batchSize=100
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model=MainModel()
    N=20
    best_test_loss = 50 # start saving after this threshold
    best_endpoint_loss = 50
    for epo in range(epochs):
            #print(f"Epoch : {epo+1}")
            for it in range(int(len(train_dataset)/batchSize)):
                trajx_train = next_batch(train_dataset,batchSize)
                trajx_test = next_batch(test_dataset,len(test_dataset))
                
                train_loss, rcl, kld, adl = train(train_dataset, model,optimizer)
                test_loss, final_point_loss_best, final_point_loss_avg = test(trajx_test, model, best_of_n = N)
                
                if best_test_loss > test_loss:
                    print("Epoch: ", epo+1)
                    print('################## BEST PERFORMANCE {:0.2f} ########'.format(test_loss))
                    best_test_loss = test_loss
                """"
                if best_test_loss < 10.25:
                    save_path = './content/trained.pt'
                """

                if final_point_loss_best < best_endpoint_loss:
                    best_endpoint_loss = final_point_loss_best

                print("Train Loss", train_loss)
                print("RCL", rcl)
                print("KLD", kld)
                print("ADL", adl)
                print("Test ADE", test_loss)
                print("Test Average FDE (Across  all samples)", final_point_loss_avg)
                print("Test Min FDE", final_point_loss_best)
                print("Test Best ADE Loss So Far (N = {})".format(N), best_test_loss)
                print("Test Best Min FDE (N = {})".format(N), best_endpoint_loss)

In [11]:
run_train()

Epoch:  1
################## BEST PERFORMANCE 4.98 ########
Train Loss tf.Tensor(3048345.0, shape=(), dtype=float32)
RCL tf.Tensor(422645.8, shape=(), dtype=float32)
KLD tf.Tensor(2409474.0, shape=(), dtype=float32)
ADL tf.Tensor(216225.27, shape=(), dtype=float32)
Test ADE 4.982340326296757
Test Average FDE (Across  all samples) 6.504974006204044
Test Min FDE 6.504766486672795
Test Best ADE Loss So Far (N = 20) 4.982340326296757
Test Best Min FDE (N = 20) 6.504766486672795
Epoch:  1
################## BEST PERFORMANCE 4.98 ########
Train Loss tf.Tensor(1425791.0, shape=(), dtype=float32)
RCL tf.Tensor(422138.1, shape=(), dtype=float32)
KLD tf.Tensor(787514.4, shape=(), dtype=float32)
ADL tf.Tensor(216138.52, shape=(), dtype=float32)
Test ADE 4.980262151206231
Test Average FDE (Across  all samples) 6.494419232536765
Test Min FDE 6.494060202205882
Test Best ADE Loss So Far (N = 20) 4.980262151206231
Test Best Min FDE (N = 20) 6.494060202205882
Epoch:  1
################## BEST PERFORMAN

Epoch:  1
################## BEST PERFORMANCE 4.64 ########
Train Loss tf.Tensor(574361.0, shape=(), dtype=float32)
RCL tf.Tensor(351473.72, shape=(), dtype=float32)
KLD tf.Tensor(16491.398, shape=(), dtype=float32)
ADL tf.Tensor(206395.9, shape=(), dtype=float32)
Test ADE 4.640228967279596
Test Average FDE (Across  all samples) 5.077940099379596
Test Min FDE 5.073987534466911
Test Best ADE Loss So Far (N = 20) 4.640228967279596
Test Best Min FDE (N = 20) 5.073987534466911
Epoch:  1
################## BEST PERFORMANCE 4.58 ########
Train Loss tf.Tensor(560233.9, shape=(), dtype=float32)
RCL tf.Tensor(340807.5, shape=(), dtype=float32)
KLD tf.Tensor(14729.061, shape=(), dtype=float32)
ADL tf.Tensor(204697.31, shape=(), dtype=float32)
Test ADE 4.578807769179716
Test Average FDE (Across  all samples) 4.825107350068934
Test Min FDE 4.8205473058363975
Test Best ADE Loss So Far (N = 20) 4.578807769179716
Test Best Min FDE (N = 20) 4.8205473058363975
Epoch:  1
################## BEST PERFORMA

Epoch:  1
################## BEST PERFORMANCE 2.82 ########
Train Loss tf.Tensor(414382.78, shape=(), dtype=float32)
RCL tf.Tensor(264944.44, shape=(), dtype=float32)
KLD tf.Tensor(4631.555, shape=(), dtype=float32)
ADL tf.Tensor(144806.78, shape=(), dtype=float32)
Test ADE 2.8177078475528816
Test Average FDE (Across  all samples) 3.6575547162224264
Test Min FDE 3.6430929744944853
Test Best ADE Loss So Far (N = 20) 2.8177078475528816
Test Best Min FDE (N = 20) 3.580331959443934
Train Loss tf.Tensor(409988.7, shape=(), dtype=float32)
RCL tf.Tensor(262506.34, shape=(), dtype=float32)
KLD tf.Tensor(4475.3535, shape=(), dtype=float32)
ADL tf.Tensor(143007.02, shape=(), dtype=float32)
Test ADE 2.843775743659335
Test Average FDE (Across  all samples) 3.7889723833869486
Test Min FDE 3.773984303193934
Test Best ADE Loss So Far (N = 20) 2.8177078475528816
Test Best Min FDE (N = 20) 3.580331959443934
Train Loss tf.Tensor(407674.62, shape=(), dtype=float32)
RCL tf.Tensor(261276.3, shape=(), dtype

Train Loss tf.Tensor(375267.25, shape=(), dtype=float32)
RCL tf.Tensor(242993.92, shape=(), dtype=float32)
KLD tf.Tensor(2971.9968, shape=(), dtype=float32)
ADL tf.Tensor(129301.33, shape=(), dtype=float32)
Test ADE 2.602648683683931
Test Average FDE (Across  all samples) 3.6249838436351105
Test Min FDE 3.605517578125
Test Best ADE Loss So Far (N = 20) 2.585004777247689
Test Best Min FDE (N = 20) 3.580331959443934
Epoch:  1
################## BEST PERFORMANCE 2.58 ########
Train Loss tf.Tensor(374659.25, shape=(), dtype=float32)
RCL tf.Tensor(243090.75, shape=(), dtype=float32)
KLD tf.Tensor(2909.1272, shape=(), dtype=float32)
ADL tf.Tensor(128659.37, shape=(), dtype=float32)
Test ADE 2.5796544342932823
Test Average FDE (Across  all samples) 3.620281982421875
Test Min FDE 3.600750732421875
Test Best ADE Loss So Far (N = 20) 2.5796544342932823
Test Best Min FDE (N = 20) 3.580331959443934
Epoch:  1
################## BEST PERFORMANCE 2.56 ########
Train Loss tf.Tensor(374035.8, shape=(),

Epoch:  1
################## BEST PERFORMANCE 2.49 ########
Train Loss tf.Tensor(369202.44, shape=(), dtype=float32)
RCL tf.Tensor(240933.4, shape=(), dtype=float32)
KLD tf.Tensor(2017.6411, shape=(), dtype=float32)
ADL tf.Tensor(126251.375, shape=(), dtype=float32)
Test ADE 2.4876223446920482
Test Average FDE (Across  all samples) 3.5941147748161764
Test Min FDE 3.5723812327665443
Test Best ADE Loss So Far (N = 20) 2.4876223446920482
Test Best Min FDE (N = 20) 3.5723812327665443
Train Loss tf.Tensor(369088.28, shape=(), dtype=float32)
RCL tf.Tensor(240899.62, shape=(), dtype=float32)
KLD tf.Tensor(1996.0411, shape=(), dtype=float32)
ADL tf.Tensor(126192.6, shape=(), dtype=float32)
Test ADE 2.4906787550409217
Test Average FDE (Across  all samples) 3.581633444393382
Test Min FDE 3.559505687040441
Test Best ADE Loss So Far (N = 20) 2.4876223446920482
Test Best Min FDE (N = 20) 3.559505687040441
Train Loss tf.Tensor(368968.56, shape=(), dtype=float32)
RCL tf.Tensor(240868.36, shape=(), dt

...

Train Loss tf.Tensor(357723.44, shape=(), dtype=float32)
RCL tf.Tensor(234147.3, shape=(), dtype=float32)
KLD tf.Tensor(22.20153, shape=(), dtype=float32)
ADL tf.Tensor(123553.94, shape=(), dtype=float32)
Test ADE 2.443115573587479
Test Average FDE (Across  all samples) 3.5889573041130514
Test Min FDE 3.5503191779641545
Test Best ADE Loss So Far (N = 20) 2.387388422755562
Test Best Min FDE (N = 20) 3.3757895076976103
Train Loss tf.Tensor(357630.88, shape=(), dtype=float32)
RCL tf.Tensor(234082.94, shape=(), dtype=float32)
KLD tf.Tensor(22.098057, shape=(), dtype=float32)
ADL tf.Tensor(123525.86, shape=(), dtype=float32)
Test ADE 2.407203471269505
Test Average FDE (Across  all samples) 3.4958636115579043
Test Min FDE 3.457830810546875
Test Best ADE Loss So Far (N = 20) 2.387388422755562
Test Best Min FDE (N = 20) 3.3757895076976103
Train Loss tf.Tensor(357538.56, shape=(), dtype=float32)
RCL tf.Tensor(234008.31, shape=(), dtype=float32)
KLD tf.Tensor(22.00684, shape=(), dtype=float32)
A

Train Loss tf.Tensor(358497.06, shape=(), dtype=float32)
RCL tf.Tensor(234738.53, shape=(), dtype=float32)
KLD tf.Tensor(20.246609, shape=(), dtype=float32)
ADL tf.Tensor(123738.3, shape=(), dtype=float32)
Test ADE 2.413917889672103
Test Average FDE (Across  all samples) 3.4754376579733455
Test Min FDE 3.4366462258731616
Test Best ADE Loss So Far (N = 20) 2.387388422755562
Test Best Min FDE (N = 20) 3.3757895076976103
Train Loss tf.Tensor(358866.97, shape=(), dtype=float32)
RCL tf.Tensor(235004.17, shape=(), dtype=float32)
KLD tf.Tensor(20.190681, shape=(), dtype=float32)
ADL tf.Tensor(123842.61, shape=(), dtype=float32)
Test ADE 2.492168586592169
Test Average FDE (Across  all samples) 3.6754645852481618
Test Min FDE 3.639251349954044
Test Best ADE Loss So Far (N = 20) 2.387388422755562
Test Best Min FDE (N = 20) 3.3757895076976103
Train Loss tf.Tensor(359166.28, shape=(), dtype=float32)
RCL tf.Tensor(235240.33, shape=(), dtype=float32)
KLD tf.Tensor(20.075535, shape=(), dtype=float32)

Train Loss tf.Tensor(357514.62, shape=(), dtype=float32)
RCL tf.Tensor(233999.39, shape=(), dtype=float32)
KLD tf.Tensor(18.700481, shape=(), dtype=float32)
ADL tf.Tensor(123496.53, shape=(), dtype=float32)
Test ADE 2.422555646725738
Test Average FDE (Across  all samples) 3.530105411305147
Test Min FDE 3.4914464613970586
Test Best ADE Loss So Far (N = 20) 2.387388422755562
Test Best Min FDE (N = 20) 3.3757895076976103
Train Loss tf.Tensor(357439.3, shape=(), dtype=float32)
RCL tf.Tensor(233932.81, shape=(), dtype=float32)
KLD tf.Tensor(18.623644, shape=(), dtype=float32)
ADL tf.Tensor(123487.89, shape=(), dtype=float32)
Test ADE 2.4376416190801993
Test Average FDE (Across  all samples) 3.576598762063419
Test Min FDE 3.538689467486213
Test Best ADE Loss So Far (N = 20) 2.387388422755562
Test Best Min FDE (N = 20) 3.3757895076976103
Train Loss tf.Tensor(357691.06, shape=(), dtype=float32)
RCL tf.Tensor(234137.45, shape=(), dtype=float32)
KLD tf.Tensor(18.516817, shape=(), dtype=float32)


Train Loss tf.Tensor(357640.12, shape=(), dtype=float32)
RCL tf.Tensor(234091.17, shape=(), dtype=float32)
KLD tf.Tensor(17.092962, shape=(), dtype=float32)
ADL tf.Tensor(123531.84, shape=(), dtype=float32)
Test ADE 2.4449785773569053
Test Average FDE (Across  all samples) 3.5844521915211396
Test Min FDE 3.5472821403952204
Test Best ADE Loss So Far (N = 20) 2.387388422755562
Test Best Min FDE (N = 20) 3.3757895076976103
Train Loss tf.Tensor(357464.66, shape=(), dtype=float32)
RCL tf.Tensor(233953.06, shape=(), dtype=float32)
KLD tf.Tensor(17.023811, shape=(), dtype=float32)
ADL tf.Tensor(123494.56, shape=(), dtype=float32)
Test ADE 2.4166713573178735
Test Average FDE (Across  all samples) 3.5227887321920957
Test Min FDE 3.4834422392003677
Test Best ADE Loss So Far (N = 20) 2.387388422755562
Test Best Min FDE (N = 20) 3.3757895076976103
Train Loss tf.Tensor(357313.3, shape=(), dtype=float32)
RCL tf.Tensor(233841.81, shape=(), dtype=float32)
KLD tf.Tensor(16.945816, shape=(), dtype=float

KeyboardInterrupt: 